In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install segmentation-models-pytorch
!pip install torchmetrics
!pip install wandb

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=a45718199a6e0ab70af33b4628d3fef5127cfd99f08c47ad2e76f66c1c365047
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=107c41b1348bd586728fb0c11a3dbb735ffcaa210569f5b0de0be3927bb622a1
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch

In [3]:
import numpy as np
from tqdm.notebook import tqdm
import scipy
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize
import os
from torchmetrics import JaccardIndex
from torchmetrics.detection import IntersectionOverUnion
from segmentation_models_pytorch.losses import DiceLoss
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
import os.path as osp
from torchvision import models


import wandb

!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import os
import os.path as osp

def get_matching_files(data_dir, phase):
    # List all image and mask files
    img_dir = osp.join(data_dir, phase, 'img')
    mask_dir = osp.join(data_dir, phase, 'mask')

    # Get list of all files in the img and mask directories
    img_files = sorted(os.listdir(img_dir))
    mask_files = sorted(os.listdir(mask_dir))

    # Initialize lists to store matched image-mask pairs
    image_list = []
    mask_list = []

    # Iterate through image files and find corresponding mask
    for img_file in img_files:
        # Base name without extension
        base_name = img_file.split('.')[0]

        # Try to find a matching mask with the same base name
        matching_mask = next((m for m in mask_files if m.startswith(base_name)), None)

        if matching_mask:
            # Add the full path for both image and mask
            image_list.append(osp.join(img_dir, img_file))
            mask_list.append(osp.join(mask_dir, matching_mask))
        else:
            print(f"Warning: No matching mask found for image {img_file}")

    return image_list, mask_list


In [5]:
class OCTDataset(Dataset):
    def __init__(self, data_dir, phase, transforms):
        self.data_dir = data_dir
        self.phase = phase
        self.transforms = transforms
        self.image_list = None
        self.label_list = None
        self.read_lists()

    def __getitem__(self, index):
        image_path = self.image_list[index]
        mask_path = self.label_list[index]

        # Debugging: Ensure the pairing is correct
        # print(f"Image: {image_path}, Mask: {mask_path}")

        image = Image.open(image_path).convert('RGB')
        label = Image.open(mask_path)

        # Apply transformations
        data = list(self.transforms(image, label))
        image = data[0]
        label = data[1]

        return image, label.long()

    def __len__(self):
        return len(self.image_list)

    def read_lists(self):
        # Get matching image and mask files
        self.image_list, self.label_list = get_matching_files(self.data_dir, self.phase)
        print(f"Total number of {self.phase} images: {len(self.image_list)}")


In [6]:
import numpy as np
import torch
from PIL import Image

class Label_Transform(object):
    def __init__(self, label_pixel=(26, 51, 77, 102, 128, 153, 179, 204, 230, 255)):
        self.label_pixel = label_pixel

    def __call__(self, image, label, *args):
        label = np.array(label)
        for i in range(len(self.label_pixel)):
            label[label == self.label_pixel[i]] = i+1

        # Ensure label array is of type uint8 and then convert to tensor
        label = label.astype(np.uint8)
        return image, torch.tensor(label, dtype=torch.long)


class Normalize(object):
    """Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor, i.e.
    channel = (channel - mean) / std
    """

    def __init__(self, mean, std):
        self.mean = torch.FloatTensor(mean)
        self.std = torch.FloatTensor(std)

    def __call__(self, image, label=None):
        for t, m, s in zip(image, self.mean, self.std):
            t.sub_(m).div_(s)
        if label is None:
            return image,
        else:
            return image, label

class ToTensor(object):
    """Converts a PIL.Image or numpy.ndarray (H x W x C) in the range
    [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
    """

    def __call__(self, pic, label=None):
        if isinstance(pic, np.ndarray):
            # handle numpy array
            img = torch.from_numpy(pic)
        else:
            # handle PIL Image
            img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
            nchannel = len(pic.mode)
            img = img.view(pic.size[1], pic.size[0], nchannel)
            img = img.transpose(0, 1).transpose(0, 2).contiguous()
        img = img.float().div(255)
        if label is None:
            return img,
        else:
            return img, label

class Resize(object):
    def __init__(self, size):
        self.size = size  # Tuple (width, height) for the new size

    def __call__(self, image, label=None):
        image = image.resize(self.size, Image.BILINEAR)  # Resize image using bilinear interpolation
        if label is not None:
            label = label.resize(self.size, Image.NEAREST)  # Resize mask using nearest-neighbor to preserve labels
        if label is None:
            return image,
        else:
            return image, label



class Compose(object):
    """Composes several transforms together.
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, *args):
        for t in self.transforms:
            args = t(*args)
        return args


In [7]:
from torch.utils.data import Subset
transforms = Compose([
    Resize((512, 512)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    Label_Transform(),

])


train_dataset = OCTDataset(data_dir = 'drive/My Drive/oct_dataset', phase = 'train',transforms = transforms)
val_dataset = OCTDataset(data_dir = 'drive/My Drive/oct_dataset', phase = 'eval', transforms = transforms)
test_dataset = OCTDataset(data_dir = 'drive/My Drive/oct_dataset', phase = 'test', transforms = transforms)

#Choosing different sizes of dataset
train_size = len(train_dataset)


indices = list(range(train_size))
np.random.seed(42)  # Ensure reproducibility
# np.random.shuffle(indices)
split = int(np.floor(1 * train_size))  # Here is where u change the amount of data
train_indices = indices[:split]

# Use Subset to create a dataset with only the selected indices
train_dataset = Subset(train_dataset, train_indices)

print(len(train_dataset))


batch_size = 16

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

#Example to see what the data shape is
for images, labels in train_loader:
  print(images.shape)
  print(labels.shape)
  break



Total number of train images: 148
Total number of eval images: 48
Total number of test images: 48
148


<ipython-input-6-52f76de3051b>:48: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))


torch.Size([16, 3, 512, 512])
torch.Size([16, 512, 512])


In [8]:
import segmentation_models_pytorch as smp
# # Model
# class ResNetDeepLabV3(nn.Module):
#     def __init__(self, classes):
#         super(ResNetDeepLabV3, self).__init__()
#         self.classes = classes  # Store the number of classes
#         self.model = smp.DeepLabV3(
#             encoder_name="resnet50",
#             encoder_weights= "imagenet",
#             in_channels = 3,
#             classes=self.classes,  # Set the number of classes
#             activation=None
#         )

#     def forward(self, x):
#         return self.model(x)

# model = ResNetDeepLabV3(classes = 11)

In [9]:
import torch
import segmentation_models_pytorch as smp

def load_custom_weights_unet(model, custom_weights_path):
    # Load the custom weights
    custom_weights = torch.load(custom_weights_path)

    # Get the state dict of the encoder
    encoder_state_dict = model.encoder.state_dict()

    # Create a new state dict for the mapped weights
    new_state_dict = {}

    # Define a mapping between custom weight keys and encoder keys
    key_mapping = {
        '0': 'firstconv',
        '1': 'firstbn',
        '4.0': 'layer1.0',
        '4.1': 'layer1.1',
        '4.2': 'layer1.2',
        '5.0': 'layer2.0',
        '5.1': 'layer2.1',
        '5.2': 'layer2.2',
        '5.3': 'layer2.3',
        '6.0': 'layer3.0',
        '6.1': 'layer3.1',
        '6.2': 'layer3.2',
        '6.3': 'layer3.3',
        '6.4': 'layer3.4',
        '6.5': 'layer3.5',
        '7.0': 'layer4.0',
        '7.1': 'layer4.1',
        '7.2': 'layer4.2',
    }

    # Map the custom weights to the encoder structure
    for k, v in custom_weights['model'].items():
        for custom_key, encoder_key in key_mapping.items():
            if k.startswith(custom_key):
                new_key = k.replace(custom_key, encoder_key, 1)
                if new_key in encoder_state_dict:
                    new_state_dict[new_key] = v

    # Load the mapped weights into the encoder
    model.encoder.load_state_dict(new_state_dict, strict=False)

    return model

# Create a U-Net model with a ResNet50 encoder
model = smp.DeepLabV3(
    encoder_name="resnet50",
    encoder_weights=None,  # We'll load custom weights
    in_channels=3,
    classes=11
)

# Load custom pretrained weights for the ResNet50 encoder
custom_weights_path = 'drive/My Drive/Colab Notebooks/SimCLR/No-Crop/best.pth'
model = load_custom_weights_unet(model, custom_weights_path)

# Optionally move the model to GPU if needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print("Custom weights loaded successfully!")

<ipython-input-9-5eea6aedef65>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  custom_weights = torch.load(custom_weights_path)


Custom weights loaded successfully!


In [10]:
from torchmetrics.classification import Dice, JaccardIndex
import torch.nn.functional as F
class DiceLoss(nn.Module):
    def __init__(self, eps=1e-7, activation='softmax'):
        super().__init__()
        self.activation = activation
        self.eps = eps

    def forward(self, y_pr, y_gt):
        if self.activation == 'softmax':
            y_pr = F.softmax(y_pr, dim=1)
        elif self.activation == 'sigmoid':
            y_pr = torch.sigmoid(y_pr)

        num_classes = y_pr.shape[1]
        y_gt = F.one_hot(y_gt, num_classes=num_classes).permute(0, 3, 1, 2).float()

        intersection = torch.sum(y_pr * y_gt, dim=[0, 2, 3])
        union = torch.sum(y_pr, dim=[0, 2, 3]) + torch.sum(y_gt, dim=[0, 2, 3])

        dice = (2.0 * intersection + self.eps) / (union + self.eps)
        dice_loss = 1.0 - torch.mean(dice)

        return dice_loss


class CEDiceLoss(nn.Module):
    def __init__(self, eps=1e-7, activation='softmax', lambda_dice=1.0, lambda_ce=1.0):
        super().__init__()
        self.dice_loss = DiceLoss(eps, activation)
        self.ce_loss = nn.CrossEntropyLoss(reduction='mean')
        self.lambda_dice = lambda_dice
        self.lambda_ce = lambda_ce

    def forward(self, y_pr, y_gt):
        dice = self.dice_loss(y_pr, y_gt)
        ce = self.ce_loss(y_pr, y_gt)
        return self.lambda_dice * dice + self.lambda_ce * ce

# Metrics
dice_coef = Dice(average='micro')
iou = JaccardIndex(task="multiclass", num_classes=11)

In [11]:
# Initialize W&B
# Initialize W&B
wandb.init(
    project="Final_Research",
    config={
        "learning_rate": 1e-3,
        "data size" : 148
    },
    name="DeepLabv3 SimCLR 100"
    )


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 2455744 (2455744-university-of-witwatersrand). Use `wandb login --relogin` to force relogin


In [12]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
import wandb
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
from torchmetrics import Accuracy


def train(model, train_loader, val_loader, epochs, loss_fn, optimizer, device):


    best_val_loss = float('inf')
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
    patience = 20 # Early stopping patience
    patience_counter = 0

    # Initialize metrics
    dice_metric = Dice(average='micro').to(device)
    iou_metric = JaccardIndex(task="multiclass", num_classes=11).to(device)
    accuracy_metric = Accuracy(task="multiclass", num_classes=11).to(device)

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        train_dice = 0.0
        train_iou = 0.0
        train_acc = 0.0

        for data, target in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            data, target = data.float().to(device), target.to(device)
            optimizer.zero_grad()
            # output = model(data)
            output = model(data)

            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            with torch.no_grad():
                pred_labels = torch.argmax(output, dim=1)
                train_dice += dice_metric(pred_labels, target)
                train_iou += iou_metric(pred_labels, target)
                train_acc += accuracy_metric(pred_labels, target)
        #Averages
        train_loss /= len(train_loader)
        train_dice /= len(train_loader)
        train_iou /= len(train_loader)
        train_acc /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        val_dice = 0.0
        val_iou = 0.0
        val_acc =0.0
        with torch.no_grad():
            for data, target, *_ in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
                data, target = data.float().to(device), target.to(device)
                # output = model(data)
                output = model(data)

                loss = loss_fn(output, target)
                val_loss += loss.item()

                pred_labels = torch.argmax(output, dim=1)
                val_dice += dice_metric(pred_labels, target)
                val_iou += iou_metric(pred_labels, target)
                val_acc += accuracy_metric(pred_labels, target)

        val_loss /= len(val_loader)
        val_dice /= len(val_loader)
        val_iou /= len(val_loader)
        val_acc /= len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Train Dice: {train_dice:.4f}, Train IoU: {train_iou:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Dice: {val_dice:.4f}, Val IoU: {val_iou:.4f}")

        # Learning rate scheduling
        scheduler.step(val_loss)

        # Model checkpointing
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'drive/MyDrive/Colab Notebooks/DeepLabV3 SimCLR/Best_model_100.pth')
            patience_counter = 0
        else:
            patience_counter += 1

        # Early stopping
        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch + 1} epochs")
            break

        # Log metrics to W&B
        wandb.log({
            "Epoch": epoch + 1,
            "Train Loss": train_loss,
            "Train Dice": train_dice,
            "Train IoU": train_iou,
            "Train Accuracy": train_acc,
            "Val Loss": val_loss,
            "Val Dice": val_dice,
            "Val IoU": val_iou,
            "Val Accuracy": val_acc,
            "Learning Rate": optimizer.param_groups[0]['lr']
        })

    # wandb.finish()
    return model

In [13]:
import torch
import numpy as np
import matplotlib.pyplot as plt

def visualize_results(model, data_loader, device, num_samples=5, class_colors=None):
    model.eval()
    images, targets, predictions = [], [], []

    with torch.no_grad():
        for data, target, *_ in data_loader:
            if len(images) >= num_samples:
                break
            data, target = data.float().to(device), target.long().to(device)
            output = model(data)
            pred = torch.argmax(output, dim=1)

            images.extend(data.cpu())
            targets.extend(target.cpu())
            predictions.extend(pred.cpu())

    images = images[:num_samples]
    targets = targets[:num_samples]
    predictions = predictions[:num_samples]

    fig, axs = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))

    for i in range(num_samples):
        # Original Image
        img = images[i].permute(1, 2, 0).numpy()
        if img.min() < 0 or img.max() > 1:  # Check if image needs normalization adjustment
            img = (img - img.min()) / (img.max() - img.min())  # Normalize to [0, 1]
        axs[i, 0].imshow(img)
        axs[i, 0].set_title('Original Image')
        axs[i, 0].axis('off')

        # Ground Truth
        gt_mask = targets[i].squeeze().numpy()
        if class_colors:
            gt_colored = np.zeros((gt_mask.shape[0], gt_mask.shape[1], 3), dtype=np.uint8)
            for class_idx, color in enumerate(class_colors):
                gt_colored[gt_mask == class_idx] = color
            axs[i, 1].imshow(gt_colored)
        else:
            axs[i, 1].imshow(gt_mask, cmap='gray')  # Use grayscale if no colors provided
        axs[i, 1].set_title('Ground Truth')
        axs[i, 1].axis('off')

        # Prediction
        pred_mask = predictions[i].squeeze().numpy()
        if class_colors:
            pred_colored = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8)
            for class_idx, color in enumerate(class_colors):
                pred_colored[pred_mask == class_idx] = color
            axs[i, 2].imshow(pred_colored)
        else:
            axs[i, 2].imshow(pred_mask, cmap='gray')  # Use grayscale if no colors provided
        axs[i, 2].set_title('Prediction')
        axs[i, 2].axis('off')

    plt.tight_layout()
    return fig


In [14]:
# Define your model, loss function, optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
loss_fn = CEDiceLoss(lambda_dice=0, lambda_ce=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train the model
trained_model = train(model, train_loader, val_loader, epochs=100, loss_fn=loss_fn, optimizer=optimizer, device=device)

# Define class colors (adjust according to your classes)
class_colors = [
    [0, 0, 0],        # Class 0 (black)
    [255, 0, 0],      # Class 1 (red)
    [0, 255, 0],      # Class 2 (green)
    [0, 0, 255],      # Class 3 (blue)
    [255, 255, 0],    # Class 4 (yellow)
    [255, 0, 255],    # Class 5 (magenta)
    [0, 255, 255],    # Class 6 (cyan)
    [192, 192, 192],  # Class 7 (silver)
    [128, 0, 0],      # Class 8 (maroon)
    [128, 128, 0],    # Class 9 (olive)
    [0, 128, 0]       # Class 10 (dark green)
]

# Visualize results after training
# fig = visualize_results(trained_model, val_loader, device, num_samples=5, class_colors=class_colors)
# plt.show()

# # Log the visualization to W&B
# wandb.init(project="Final_Research", name="Post_Training_Visualisation_DeepLabv3_ImageNet_100")
# wandb.log({"Validation Predictions DeepLabv3 ImageNet 100": wandb.Image(fig)})


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Epoch 1/100 - Validation: 100%|██████████| 3/3 [01:09<00:00, 23.10s/it]


Epoch 1: Train Loss: 1.3405, Train Dice: 0.7644, Train IoU: 0.1434, Val Loss: 0.9948, Val Dice: 0.8721, Val IoU: 0.0793


Epoch 2/100 - Validation: 100%|██████████| 3/3 [00:03<00:00,  1.01s/it]


Epoch 2: Train Loss: 0.4701, Train Dice: 0.9175, Train IoU: 0.2297, Val Loss: 0.6540, Val Dice: 0.8721, Val IoU: 0.0793


Epoch 3/100 - Validation: 100%|██████████| 3/3 [00:02<00:00,  1.02it/s]


...

Epoch 45: Train Loss: 0.0549, Train Dice: 0.9797, Train IoU: 0.7227, Val Loss: 0.1337, Val Dice: 0.9594, Val IoU: 0.6050


Epoch 46/100 - Validation: 100%|██████████| 3/3 [00:02<00:00,  1.03it/s]


Epoch 46: Train Loss: 0.0556, Train Dice: 0.9795, Train IoU: 0.7214, Val Loss: 0.1338, Val Dice: 0.9594, Val IoU: 0.6050


Epoch 47/100 - Validation: 100%|██████████| 3/3 [00:02<00:00,  1.00it/s]

Epoch 47: Train Loss: 0.0555, Train Dice: 0.9795, Train IoU: 0.7208, Val Loss: 0.1338, Val Dice: 0.9594, Val IoU: 0.6047
Early stopping triggered after 47 epochs


In [ ]:
from tqdm import tqdm

def test(model, test_loader, loss_fn, device):
    model.eval()  # Set model to evaluation mode
    test_loss = 0.0
    test_dice = 0.0
    test_iou = 0.0
    test_acc =0.0

    dice_metric = Dice(average='micro').to(device)
    iou_metric = JaccardIndex(task="multiclass", num_classes=11).to(device)  # Adjust num_classes as needed
    accuracy_metric = Accuracy(task="multiclass", num_classes=11).to(device)
    with torch.no_grad():  # Disable gradient computation
        for data, target, *_ in tqdm(test_loader, desc="Testing"):
            data, target = data.float().to(device), target.to(device)

            # Forward pass
            output = model(data)
            loss = loss_fn(output, target)
            test_loss += loss.item()

            # Get predictions
            pred_labels = torch.argmax(output, dim=1)

            # Calculate metrics
            test_dice += dice_metric(pred_labels, target)
            test_iou += iou_metric(pred_labels, target)
            test_acc += accuracy_metric(pred_labels, target)

    # Average metrics over all batches
    test_loss /= len(test_loader)
    test_dice /= len(test_loader)
    test_iou /= len(test_loader)
    test_acc /= len(test_loader)

    print(f"Test Loss: {test_loss:.4f}, Test Dice: {test_dice:.4f}, Test IoU: {test_iou:.4f}")

    return test_loss, test_dice, test_iou, test_acc

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = DeepLabV3(classes = 11).to(device)
model.load_state_dict(torch.load('drive/MyDrive/Colab Notebooks/DeepLabV3 SimCLR/Best_model_100.pth'))
loss_fn = CEDiceLoss(lambda_dice=0, lambda_ce=1)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Run the test function
test_loss, test_dice, test_iou, test_acc = test(model, test_loader, loss_fn, device)

# Optionally, log test results to W&B
wandb.init(project="Final_Research", name="Test_Results_DeepLabv3_SimCLR_100")
wandb.log({
    "Test Loss": test_loss,
    "Test Dice": test_dice,
    "Test IoU": test_iou,
    "Test Accuracy": test_acc
})
wandb.finish()

# fig = visualize_results(trained_model, test_loader, device, num_samples=5, class_colors=class_colors)
# plt.show()

# # Log the visualization to W&B
# wandb.init(project="Final_Research", name="Test_Visualization_DeepLabv3_ImageNet_10")
# wandb.log({"Final Predictions: DeepLabv3 ImageNet 100": wandb.Image(fig)})
# wandb.finish()

<ipython-input-15-3175d33043ad>:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('drive/MyDrive/Colab Notebooks/DeepLabV3 SimCLR/Best_model